# BSEP 

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolDescriptors
from rdkit.Chem import PandasTools as PandasTools
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from standardise import Standardization

## Step 1: Data Collection

### UNIVIE Data 

In [ ]:
Univie_Data = "data/BSEP_Univie.sdf"

### Intern Data*

In [ ]:
# Please add the name of your file
Intern_Data = "data/BSEP_ChEMBL28.sdf"

### Load SDF File

In [ ]:
df_Univie_Data = PandasTools.LoadSDF(Univie_Data)
df_Intern_Data = PandasTools.LoadSDF(Intern_Data)

### InChIs, SMILES & InChIkey Calculation

In [ ]:
df_Intern_Data['InChIs'] = df_Intern_Data['ROMol'].map(lambda x:AllChem.MolToInchi(x))
df_Intern_Data['SMILES'] = df_Intern_Data['ROMol'].map(lambda x:AllChem.MolToSmiles(x))
df_Intern_Data['InChIKey'] = df_Intern_Data['ROMol'].map(lambda x:AllChem.MolToInchiKey(x))

### Remove Stereochemistry

In [ ]:
ls_InChIs = df_Intern_Data['InChIs'].tolist()

In [ ]:
def removeStereo(fullInchi, position, delimiter):
    return delimiter.join(fullInchi.split(delimiter)[:position])

ls_Curated_InChIs = []

for i in ls_InChIs:
    ls_noniso = (removeStereo(i,4,"/"))
    ls_Curated_InChIs.append(ls_noniso)
    
df_Intern_Data['Curated_InChIs'] = ls_Curated_InChIs

### Duplicate Check

In [ ]:
Intern_Unique = df_Intern_Data.drop_duplicates(subset='Curated_InChIs',keep='first')
Intern_Duplicates = df_Intern_Data[df_Intern_Data.duplicated(['Curated_InChIs'], keep='first')]

### Comparison of Classification Values of Removed Duplicates

In [ ]:
def remove_classerror(x):
        if x in df_classerror:
            df_classerror.remove(x)
            return False
        return True

if len(Intern_Duplicates.index) == 0:
    print('No duplicates found.')
else:
    Comparison_dupl = Intern_Duplicates[['ROMol','Classification','Curated_InChIs']]
    Comparison_uniq = Intern_Unique[['ROMol','Classification','Curated_InChIs']]
    merged   =    pd.merge(
                  left=Comparison_dupl,
                  right=Comparison_uniq,
                  how="left",
                  left_on="Curated_InChIs",
                  right_on="Curated_InChIs")
    merged['Class_Match'] = merged.apply(lambda x : str(x.Classification_x) in str(x.Classification_y), axis=1)
    Non_Matching_Class = merged.loc[merged['Class_Match'] == False]
    df_classerror = Non_Matching_Class['Curated_InChIs'].tolist()
    Intern_Unique = Intern_Unique[Intern_Unique.Curated_InChIs.apply(remove_classerror)]
    print('Compounds with classification mismatch removed.')

### Duplicate Removal

In [ ]:
def remove_duplicates(x):
    if x in ls_Univie_Transporters:
        ls_Univie_Transporters.remove(x)
        return False
    return True

ls_Univie_Transporters = df_Univie_Data['Curated_InChIs'].tolist()


Delta_Compounds = Intern_Unique[Intern_Unique.Curated_InChIs.apply(remove_duplicates)]
Delta_Compounds.info()

### Selection of Important Columns

In [ ]:
Columns_Univie =  df_Univie_Data[['Classification', 'ROMol', 'Curated_InChIs','SMILES','InChIKey']]
Columns_Intern = Delta_Compounds[['Classification', 'ROMol', 'Curated_InChIs','SMILES','InChIKey']]
Training_Set = pd.concat([Columns_Univie, Columns_Intern], sort=False)

### Save Generated Training Set

In [ ]:
PandasTools.WriteSDF(Training_Set,"data/BSEP_Training_Set.sdf",properties=list(Training_Set.columns))

## Step 2: Data Set Preparation for ML Task

### Read SDF

#### Training set*

In [ ]:
# Please add the name of your training set
molecules = Chem.ForwardSDMolSupplier("data/BSEP_Univie.sdf", sanitize=False)

#### Test set*

In [ ]:
# Please add the name of your test set
test_molecules = Chem.ForwardSDMolSupplier("data/BSEP_ChEMBL28.sdf", sanitize=False)

### Standardisation

#### Training set

In [ ]:
stand = Standardization()

train_molcount = 0

standardised_molecules = []
            
for mol in molecules:
        train_molcount += 1

        if mol is None:
            continue

        standardisation_ok, molOrError = stand.standardise(mol)
        
        if standardisation_ok == True:
            standardised_molecules.append(molOrError)
        else:
            print(molOrError)

#### Test set

In [ ]:
stand = Standardization()

test_molcount = 0

standardised_test_molecules = []

for mol in test_molecules:
        test_molcount += 1
        
        if mol is None:
            continue
            
        standardisation_ok, molOrError = stand.standardise(mol)
        
        if standardisation_ok == True:
            standardised_test_molecules.append(molOrError)
        else:
            print(molOrError)

### Check Standardisation

In [ ]:
print (str(len(standardised_molecules)) + ' of ' + str(train_molcount) + ' compounds could be standardised.')

In [ ]:
print (str(len(standardised_test_molecules)) + ' of ' + str(test_molcount) + ' compounds could be standardised.')

### Check Descriptors

In [ ]:
from rdkit.ML.Descriptors import MoleculeDescriptors
import numpy as np

In [ ]:
with open("RDKIT_Descriptors.txt", "r") as f:
    Descriptors = []
    for descriptor in f.readlines():
        items = descriptor.rstrip('\n').rstrip(',').strip("''")
        Descriptors.append(items)

In [ ]:
print (str(len(Descriptors)) + ' descriptors are used.')

#### Training set

In [ ]:
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(Descriptors)
descriptors = []
activities = []

for mol in standardised_molecules:

    desc_np = np.asarray(calculator.CalcDescriptors(mol))

    descriptors.append(desc_np)

    activities.append(int(mol.GetProp("Classification")))

#### Test set 

In [ ]:
calculator_test = MoleculeDescriptors.MolecularDescriptorCalculator(Descriptors)
descriptors_test = []
activities_test = []

for mol in standardised_test_molecules:
    desc_np = np.asarray(calculator_test.CalcDescriptors(mol))
    descriptors_test.append(desc_np)
    activities_test.append(int(mol.GetProp("Classification")))

### Check Descriptor Calculation

#### Training set

In [ ]:
print('The descriptors for ' + str(len(descriptors)) + ' compounds could be calculated.')
print('The activities of ' + str(len(activities)) + ' compounds could be read.') 

#### Test set

In [ ]:
print('The descriptors for ' + str(len(descriptors_test)) + ' compounds could be calculated.')
print('The activities of ' + str(len(activities_test)) + ' compounds could be read.')

### Count actives / inactives

#### Training set

In [ ]:
active = 0
inactive = 0
for activity in activities:
    if activity == 1:
        active += 1
    elif activity == 0:
        inactive += 1
print('The training set includes', active, 'inhibitors and', inactive, 'non-inhibitors.')
print('The total amount of compounds in the training set is', active + inactive, '.')

#### Test set 

In [ ]:
active_test = 0
inactive_test = 0
for activity_test in activities_test:
    if activity_test == 1:
        active_test += 1
    elif activity_test == 0:
        inactive_test += 1
print('The training set includes', active_test, 'inhibitors and', inactive_test, 'non-inhibitors.')
print('The total amount of compounds in the training set is', active_test + inactive_test, '.')

#### Change NaN to 0.0

In [ ]:
df_descriptors = pd.DataFrame(descriptors)
zeros = df_descriptors.fillna(0.0)
descriptors = np.array(zeros)
descriptors.shape

## Step 3: Applicability Domain

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Add your test data for comparison*

In [ ]:
Test_Data = "data/BSEP_ChEMBL28.sdf"

In [ ]:
scaler = MinMaxScaler()
iso_map = PCA(n_components=2)
model = LocalOutlierFactor(n_neighbors=5, novelty=True, contamination=0.1, metric='euclidean')

fps = []
thresholds = []
k = 15
stand = Standardization()

scaler.fit(descriptors)
stand_desc = scaler.transform(descriptors)
iso_map.fit(stand_desc)
train_iso = iso_map.transform(stand_desc)
model.fit(train_iso)

tests = []
test_stand_desc = scaler.transform(descriptors_test)
test_iso = iso_map.transform(test_stand_desc)
#tests.append([test_iso[0][0], test_iso[0][1]])
iso_ad_pred = []
for tp in test_iso:
    iso_ad_pred.append(model.predict([tp])[0])
    tests.append([tp[0], tp[1]])


tests = np.array(tests)
xx, yy = np.meshgrid(np.linspace(-5, 5, 500), np.linspace(-5, 5, 500))
Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure()
plt.title("Applicability Domain Assesment with LOF")
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap=plt.cm.PuBu)
a = plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='darkred')
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred')
s = 40
b2 = plt.scatter(tests[:, 0], tests[:, 1], c='white', s=s,edgecolors='k', zorder=3)
plt.axis('tight')
plt.xlim((-5, 5))
plt.ylim((-5, 5))
plt.legend([a.collections[0],b2],
  ["Applicability Domain",
   "Test Set Components"],
  loc="upper left",
  prop=matplotlib.font_manager.FontProperties(size=11))
plt.ylabel("PCA Dimension 1")
plt.xlabel("PCA Dimension 0")

In [ ]:
from IPython.core.display import display, HTML

# if lof equal to 1, then it is considered as an inlier; if it is -1, then it is an outlier.
df_descriptors_test = pd.DataFrame(descriptors_test, columns = Descriptors)
df_descriptors_test['LOF'] = iso_ad_pred
Test_Outliers = df_descriptors_test[df_descriptors_test['LOF'] == -1]
New_Descriptors_test = df_descriptors_test[df_descriptors_test['LOF'] == 1]

# Read data
df_Test_Data = PandasTools.LoadSDF(Test_Data)

# Search via index outliers 
Outlier_Compounds = df_Test_Data['ROMol'].loc[Test_Outliers.index]
df_Outlier_Compounds = pd.DataFrame(Outlier_Compounds)
print(str(len(Test_Outliers)) + ' compounds have been identified as outliers in the Test Set using LOF.')

for index, row in df_Outlier_Compounds.iterrows():
    display(HTML(str(row['ROMol'])))

In [ ]:
PandasTools.WriteSDF(df_Outlier_Compounds,"results/BSEP_Outlier_Compounds.sdf",properties=list(df_Outlier_Compounds.columns))

## Step 4: Model Generation & Evaluation

### Please add the name of the used training set*

In [ ]:
SDFFile = "data/BSEP_Univie.sdf"
df_Data = PandasTools.LoadSDF(SDFFile)

### 1) Logisitic Regression (LR)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score

In [ ]:
lr_clf = LogisticRegression(solver='saga', max_iter=10000)
lr_clf.fit(descriptors,activities)

In [ ]:
import pickle

filename = 'results/BSEP_model_LR.pkl'
pickle.dump(lr_clf, open(filename, 'wb'))

#### LR -  10-CV Training Set

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

In [ ]:
accuracy_score_cv = cross_val_score(lr_clf, descriptors, activities, scoring='accuracy', cv=10, n_jobs=-1, error_score='raise')
sensitivity_score_cv = cross_val_score(lr_clf, descriptors, activities, scoring='recall', cv=10, n_jobs=-1, error_score='raise')
specificity_cv = make_scorer(recall_score, pos_label=0)
specificity_score_cv = cross_val_score(lr_clf, descriptors, activities, scoring=specificity_cv, cv=10, n_jobs=-1, error_score='raise')
balanced_accuracy_score_cv = cross_val_score(lr_clf, descriptors, activities, scoring='balanced_accuracy', cv=10, n_jobs=-1, error_score='raise' )

In [ ]:
#recall_score = cross_val_score(ANOVA_svm_model, descriptors_fs, activities, scoring='recall', cv=10, n_jobs=-1, error_score='raise')
f1_score_cv = cross_val_score(lr_clf, descriptors, activities, scoring='f1', cv=10, n_jobs=-1, error_score='raise')
auc_cv = cross_val_score(lr_clf, descriptors, activities, scoring='roc_auc', cv=10, n_jobs=-1, error_score='raise')
precision_score_cv = cross_val_score(lr_clf, descriptors, activities, scoring='precision', cv=10, n_jobs=-1, error_score='raise')
mathews_corrcoef_cv = make_scorer(matthews_corrcoef)
matthews_corrcoef_score_cv = cross_val_score(lr_clf, descriptors, activities, scoring=mathews_corrcoef_cv, cv=10, n_jobs=-1, error_score='raise')

In [ ]:
acc_lr1 = 'Accuracy: %.3f (%.3f)' % (accuracy_score_cv.mean(), accuracy_score_cv.std())
sen_lr1 = 'Sensitivity: %.3f (%.3f)' % (sensitivity_score_cv.mean(), sensitivity_score_cv.std())
spec_lr1 = 'Specificity: %.3f (%.3f)' % (specificity_score_cv.mean(), specificity_score_cv.std())
ba_lr1 = 'Balanced_Accuracy: %.3f (%.3f)' % (balanced_accuracy_score_cv.mean(), balanced_accuracy_score_cv.std())
f1_lr1 = 'F1_score %.3f (%.3f)' % (f1_score_cv.mean(), f1_score_cv.std())
auc_lr1 = 'AUC %.3f (%.3f)' % (auc_cv.mean(), auc_cv.std())
prec_lr1 = 'Precision %.3f (%.3f)' % (precision_score_cv.mean(), precision_score_cv.std())
mcc_lr1 = 'Matthews_corrcoef: %.3f (%.3f)' % (matthews_corrcoef_score_cv.mean(), matthews_corrcoef_score_cv.std())

In [ ]:
import ipywidgets as widgets

def dropdown(change):
    print(change.new)    
    
Buttons = widgets.Dropdown(
    options = [['Accuracy',acc_lr1], ['Sensitivity',sen_lr1], ['Specificity',spec_lr1], ['Balanced Accuracy',ba_lr1],['F1_score',f1_lr1], ['AUC',auc_lr1], ['Precision',prec_lr1], ['MCC',mcc_lr1]],
    description= 'Choose one:',
    disabled=False,
    indent=False
)

Buttons.observe(dropdown, names='value')
display(Buttons)

#### Wrong Predicted Compounds

In [ ]:
from IPython.core.display import display, HTML

log_predictions = lr_clf.predict(descriptors_test)
indices = [i for i in range(len(activities_test)) if activities_test[i] != log_predictions[i]]
wrong_predictions_lr = df_Test_Data.iloc[indices,:]
wrong_predicted_compounds = wrong_predictions_lr.index.tolist()
print(str(len((wrong_predicted_compounds))) + ' compounds have been predicted wrong.')
compounds_lr_wrong =  pd.DataFrame(wrong_predictions_lr['ROMol'])

for index, row in compounds_lr_wrong.iterrows():
    display(HTML(str(row['ROMol'])))

#### Save Wrong Predicted Compounds

In [ ]:
PandasTools.WriteSDF(wrong_predictions_lr,"data/BSEP_WP_LR_Compounds.sdf",properties=list(wrong_predictions_lr.columns))

#### Logistic Regression -  Evaluation Test Set

In [ ]:
lr_predict = lr_clf.predict(descriptors_test)

In [ ]:
acc_lr2 = 'Accuracy_score: %.3f' % accuracy_score(np.asarray(activities_test), lr_predict)
sen_lr2 = 'Sensitivity: %.3f ' % recall_score(np.asarray(activities_test), lr_predict)
spec_lr2 = 'Specificity: %.3f ' % recall_score(np.asarray(activities_test), lr_predict, pos_label=0)
ba_lr2 = 'Balanced_accuracy_score: %.3f' % balanced_accuracy_score(np.asarray(activities_test), lr_predict)
f1_lr2 = 'F1_score: %.3f' %  f1_score(np.asarray(activities_test), lr_predict)
auc_lr2 = 'AUC %.3f' % (roc_auc_score(np.asarray(activities_test), lr_predict))
prec_lr2 = 'Precision_score: %.3f' % precision_score(np.asarray(activities_test), lr_predict)
mcc_lr2 = 'Matthews_corrcoef: %.3f' % matthews_corrcoef(np.asarray(activities_test), lr_predict)
rec_lr2 = 'Recall_score: %.3f' % recall_score(np.asarray(activities_test), lr_predict)

In [ ]:
import ipywidgets as widgets

def dropdown(change):
    print(change.new)    
    
Buttons = widgets.Dropdown(
    options = [['Accuracy',acc_lr2], ['Sensitivity',sen_lr2], ['Specificity',spec_lr2], ['Balanced Accuracy',ba_lr2],['F1_score',f1_lr2], ['AUC',auc_lr2], ['Precision',prec_lr2], ['MCC',mcc_lr2], ['Recall',rec_lr2]],
    description= 'Choose one:',
    disabled=False,
    indent=False
)

Buttons.observe(dropdown, names='value')
display(Buttons)

In [ ]:
df_Test["Prediction"] = lr_predict
df_Test["Prediction Comparison"] = np.where(pd.to_numeric(df_Test["Classification"]) == df_Test["Prediction"], "Right", "Wrong")
df_Test["Domain"] = df_descriptors_test["LOF"]
df_Test["Domain"] = np.where(df_Test.Domain.values == 1, "In", "Out")
df_LR_Test = df_Test
df_LR_Test

In [ ]:
PandasTools.WriteSDF(df_LR_Test,"BSEP_LR_Test_Prediction.sdf",properties=list(df_LR_Test.columns))
df_LR_Test.to_csv("results/BSEP_LR_Test_Prediction.csv")

### 2) Support Vector Machine (SVM)

In [ ]:
from sklearn import svm

In [ ]:
svm_clf = svm.SVC(class_weight='balanced', C= 1.0, kernel= 'linear')
svm_clf.fit(descriptors,activities)

In [ ]:
import pickle

filename = 'results/BSEP_model_SVM.pkl'
pickle.dump(svm_clf, open(filename, 'wb'))

#### SVM  -  10-CV Training Set

In [ ]:
accuracy_score_cv = cross_val_score(svm_clf, descriptors, activities, scoring='accuracy', cv=10, n_jobs=-1, error_score='raise')
sensitivity_score_cv = cross_val_score(svm_clf, descriptors, activities, scoring='recall', cv=10, n_jobs=-1, error_score='raise')
specificity_cv = make_scorer(recall_score, pos_label=0)
specificity_score_cv = cross_val_score(svm_clf, descriptors, activities, scoring=specificity_cv, cv=10, n_jobs=-1, error_score='raise')
balanced_accuracy_score_cv = cross_val_score(svm_clf, descriptors, activities, scoring='balanced_accuracy', cv=10, n_jobs=-1, error_score='raise' )

In [ ]:
f1_score_cv = cross_val_score(svm_clf, descriptors, activities, scoring='f1', cv=10, n_jobs=-1, error_score='raise')
auc_cv = cross_val_score(svm_clf, descriptors, activities, scoring='roc_auc', cv=10, n_jobs=-1, error_score='raise')
precision_score_cv = cross_val_score(svm_clf, descriptors, activities, scoring='precision', cv=10, n_jobs=-1, error_score='raise')
mathews_corrcoef_cv = make_scorer(matthews_corrcoef)
matthews_corrcoef_score_cv = cross_val_score(svm_clf, descriptors, activities, scoring=mathews_corrcoef_cv, cv=10, n_jobs=-1, error_score='raise')

In [ ]:
acc_svm1 = 'Accuracy: %.3f (%.3f)' % (accuracy_score_cv.mean(), accuracy_score_cv.std())
sen_svm1 = 'Sensitivity: %.3f (%.3f)' % (sensitivity_score_cv.mean(), sensitivity_score_cv.std())
spec_svm1 = 'Specificity: %.3f (%.3f)' % (specificity_score_cv.mean(), specificity_score_cv.std())
ba_svm1 = 'Balanced_Accuracy: %.3f (%.3f)' % (balanced_accuracy_score_cv.mean(), balanced_accuracy_score_cv.std())
f1_svm1 = 'F1_score %.3f (%.3f)' % (f1_score_cv.mean(), f1_score_cv.std())
auc_svm1 = 'AUC %.3f (%.3f)' % (auc_cv.mean(), auc_cv.std())
prec_svm1 = 'Precision %.3f (%.3f)' % (precision_score_cv.mean(), precision_score_cv.std())
mcc_svm1 = 'Matthews_corrcoef: %.3f (%.3f)' % (matthews_corrcoef_score_cv.mean(), matthews_corrcoef_score_cv.std())

In [ ]:
import ipywidgets as widgets

def dropdown(change):
    print(change.new)    
    
Buttons = widgets.Dropdown(
    options = [['Accuracy',acc_svm1], ['Sensitivity',sen_svm1], ['Specificity',spec_svm1], ['Balanced Accuracy',ba_svm1],['F1_score',f1_svm1], ['AUC',auc_svm1], ['Precision',prec_svm1], ['MCC',mcc_svm1]],
    description= 'Choose one:',
    disabled=False,
    indent=False
)

Buttons.observe(dropdown, names='value')
display(Buttons)

#### Wrong Predicted Compounds

In [ ]:
from IPython.core.display import display, HTML

svm_predictions = svm_clf.predict(descriptors_test)
indices = [i for i in range(len(activities_test)) if activities_test[i] != svm_predictions[i]]
wrong_predictions_svm = df_Test_Data.iloc[indices,:]
wrong_predicted_compounds = wrong_predictions_svm.index.tolist()
print(str(len((wrong_predicted_compounds))) + ' compounds have been predicted wrong.')
compounds_svm_wrong =  pd.DataFrame(wrong_predictions_svm['ROMol'])

for index, row in compounds_svm_wrong.iterrows():
    display(HTML(str(row['ROMol'])))

#### Save Wrong Predicted Compounds

In [ ]:
PandasTools.WriteSDF(wrong_predictions_svm,"results/BSEP_WP_SVM_Compounds.sdf",properties=list(wrong_predictions_svm.columns))

####  SVM -  Evaluation Test  Set

In [ ]:
svm_predict = svm_clf.predict(descriptors_test)

In [ ]:
acc_svm2 = 'Accuracy_score: %.3f' % accuracy_score(np.asarray(activities_test), svm_predict)
sen_svm2 = 'Sensitivity: %.3f ' % recall_score(np.asarray(activities_test), svm_predict)
spec_svm2 = 'Specificity: %.3f ' % recall_score(np.asarray(activities_test), svm_predict, pos_label=0)
ba_svm2 = 'Balanced_accuracy_score: %.3f' % balanced_accuracy_score(np.asarray(activities_test), svm_predict)
f1_svm2 = 'F1_score: %.3f' %  f1_score(np.asarray(activities_test), svm_predict)
auc_svm2 = 'AUC %.3f' % (roc_auc_score(np.asarray(activities_test), svm_predict))
prec_svm2 = 'Precision_score: %.3f' % precision_score(np.asarray(activities_test), svm_predict)
mcc_svm2 = 'Matthews_corrcoef: %.3f' % matthews_corrcoef(np.asarray(activities_test), svm_predict)
rec_svm2 = 'Recall_score: %.3f' % recall_score(np.asarray(activities_test), svm_predict)

In [ ]:
import ipywidgets as widgets

def dropdown(change):
    print(change.new)    
    
Buttons = widgets.Dropdown(
    options = [['Accuracy',acc_svm2], ['Sensitivity',sen_svm2], ['Specificity',spec_svm2], ['Balanced Accuracy',ba_svm2],['F1_score',f1_svm2], ['AUC',auc_svm2], ['Precision',prec_svm2], ['MCC',mcc_svm2],['Recall',rec_svm2]],
    description= 'Choose one:',
    disabled=False,
    indent=False
)

Buttons.observe(dropdown, names='value')
display(Buttons)

In [ ]:
df_Test["Prediction"] = svm_predict
df_Test["Prediction Comparison"] = np.where(pd.to_numeric(df_Test["Classification"]) == df_Test["Prediction"], "Right", "Wrong")
df_Test["Domain"] = df_descriptors_test["LOF"]
df_Test["Domain"] = np.where(df_Test.Domain.values == 1, "In", "Out")
df_SVM_Test = df_Test
df_SVM_Test

In [ ]:
PandasTools.WriteSDF(df_SVM_Test,"results/BSEP_SVM_Test_Prediction.sdf",properties=list(df_SVM_Test.columns))
df_SVM_Test.to_csv("results/BSEP_SVM_Test_Prediction.csv")

### 3) Random Forest (RF)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier(class_weight="balanced", max_depth = 4, n_estimators = 10)
rf_clf.fit(descriptors, activities)

In [ ]:
import pickle

filename = 'results/BSEP_model_RF.pkl'
pickle.dump(rf_clf, open(filename, 'wb'))

#### RF - 10-CV Training Set

In [ ]:
accuracy_score_cv = cross_val_score(rf_clf, descriptors, activities, scoring='accuracy', cv=10, n_jobs=-1, error_score='raise')
sensitivity_score_cv = cross_val_score(rf_clf, descriptors, activities, scoring='recall', cv=10, n_jobs=-1, error_score='raise')
specificity_cv = make_scorer(recall_score, pos_label=0)
specificity_score_cv = cross_val_score(rf_clf, descriptors, activities, scoring=specificity_cv, cv=10, n_jobs=-1, error_score='raise')
balanced_accuracy_score_cv = cross_val_score(rf_clf, descriptors, activities, scoring='balanced_accuracy', cv=10, n_jobs=-1, error_score='raise' )

In [ ]:
f1_score_cv = cross_val_score(rf_clf, descriptors, activities, scoring='f1', cv=10, n_jobs=-1, error_score='raise')
auc_cv = cross_val_score(rf_clf, descriptors, activities, scoring='roc_auc', cv=10, n_jobs=-1, error_score='raise')
precision_score_cv = cross_val_score(rf_clf, descriptors, activities, scoring='precision', cv=10, n_jobs=-1, error_score='raise')
mathews_corrcoef_cv = make_scorer(matthews_corrcoef)
matthews_corrcoef_score_cv = cross_val_score(rf_clf, descriptors, activities, scoring=mathews_corrcoef_cv, cv=10, n_jobs=-1, error_score='raise')

In [ ]:
acc_rf1 = 'Accuracy: %.3f (%.3f)' % (accuracy_score_cv.mean(), accuracy_score_cv.std())
sen_rf1 = 'Sensitivity: %.3f (%.3f)' % (sensitivity_score_cv.mean(), sensitivity_score_cv.std())
spec_rf1 = 'Specificity: %.3f (%.3f)' % (specificity_score_cv.mean(), specificity_score_cv.std())
ba_rf1 = 'Balanced_Accuracy: %.3f (%.3f)' % (balanced_accuracy_score_cv.mean(), balanced_accuracy_score_cv.std())
f1_rf1 = 'F1_score %.3f (%.3f)' % (f1_score_cv.mean(), f1_score_cv.std())
auc_rf1 = 'AUC %.3f (%.3f)' % (auc_cv.mean(), auc_cv.std())
prec_rf1 = 'Precision %.3f (%.3f)' % (precision_score_cv.mean(), precision_score_cv.std())
mcc_rf1 = 'Matthews_corrcoef: %.3f (%.3f)' % (matthews_corrcoef_score_cv.mean(), matthews_corrcoef_score_cv.std())

In [ ]:
import ipywidgets as widgets

def dropdown(change):
    print(change.new)    
    
Buttons = widgets.Dropdown(
    options = [['Accuracy',acc_rf1], ['Sensitivity',sen_rf1], ['Specificity',spec_rf1], ['Balanced Accuracy',ba_rf1],['F1_score',f1_rf1], ['AUC',auc_rf1], ['Precision',prec_rf1], ['MCC',mcc_rf1]],
    description= 'Choose one:',
    disabled=False,
    indent=False
)

Buttons.observe(dropdown, names='value')
display(Buttons)

#### Wrong Predicted Compounds

In [ ]:
from IPython.core.display import display, HTML

rf_predictions = rf_clf.predict(descriptors_test)
indices = [i for i in range(len(activities_test)) if activities_test[i] != rf_predictions[i]]
wrong_predictions_rf = df_Test_Data.iloc[indices,:]
wrong_predicted_compounds = wrong_predictions_rf.index.tolist()
print(str(len((wrong_predicted_compounds))) + ' compounds have been predicted wrong.')
compounds_rf_wrong =  pd.DataFrame(wrong_predictions_rf['ROMol'])

for index, row in compounds_rf_wrong.iterrows():
    display(HTML(str(row['ROMol'])))

#### Save Wrong Predicted Compounds

In [ ]:
PandasTools.WriteSDF(wrong_predictions_rf,"results/BSEP_WP_RF_Compounds.sdf",properties=list(wrong_predictions_rf.columns))

#### RF - Evaluation Test Set

In [ ]:
rf_predict = rf_clf.predict(descriptors_test)

In [ ]:
acc_rf2 = 'Accuracy_score: %.3f' % accuracy_score(np.asarray(activities_test), rf_predict)
sen_rf2 = 'Sensitivity: %.3f ' % recall_score(np.asarray(activities_test), rf_predict)
spec_rf2 = 'Specificity: %.3f ' % recall_score(np.asarray(activities_test), rf_predict, pos_label=0)
ba_rf2 = 'Balanced_accuracy_score: %.3f' % balanced_accuracy_score(np.asarray(activities_test), rf_predict)
f1_rf2 = 'F1_score: %.3f' %  f1_score(np.asarray(activities_test), rf_predict)
auc_rf2 = 'AUC %.3f' % (roc_auc_score(np.asarray(activities_test), rf_predict))
prec_rf2 = 'Precision_score: %.3f' % precision_score(np.asarray(activities_test), rf_predict)
mcc_rf2 = 'Matthews_corrcoef: %.3f' % matthews_corrcoef(np.asarray(activities_test), rf_predict)
rec_rf2 = 'Recall_score: %.3f' % recall_score(np.asarray(activities_test), rf_predict)

In [ ]:
import ipywidgets as widgets

def dropdown(change):
    print(change.new)    
    
Buttons = widgets.Dropdown(
    options = [['Accuracy',acc_rf2], ['Sensitivity',sen_rf2], ['Specificity',spec_rf2], ['Balanced Accuracy',ba_rf2],['F1_score',f1_rf2], ['AUC',auc_rf2], ['Precision',prec_rf2], ['MCC',mcc_rf2],['Recall',rec_rf2]],
    description= 'Choose one:',
    disabled=False,
    indent=False
)

Buttons.observe(dropdown, names='value')
display(Buttons)

In [ ]:
df_Test["Prediction"] = rf_predict
df_Test["Prediction Comparison"] = np.where(pd.to_numeric(df_Test["Classification"]) == df_Test["Prediction"], "Right", "Wrong")
df_Test["Domain"] = df_descriptors_test["LOF"]
df_Test["Domain"] = np.where(df_Test.Domain.values == 1, "In", "Out")
df_RF_Test = df_Test
df_RF_Test

In [ ]:
PandasTools.WriteSDF(df_RF_Test,"results/BSEP_RF_Test_Prediction.sdf",properties=list(df_RF_Test.columns))
df_RF_Test.to_csv("results/BSEP_RF_Test_Prediction.csv")

### 4) K-Nearest Neighbor (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_clf = KNeighborsClassifier(metric =  'euclidean', n_neighbors = 3, weights =  'distance')
knn_clf.fit(descriptors,activities)

In [ ]:
import pickle

filename = 'results/BSEP_model_KNN.pkl'
pickle.dump(knn_clf, open(filename, 'wb'))

####  KNN -  10-CV Training Set

In [ ]:
accuracy_score_cv = cross_val_score(knn_clf, descriptors, activities, scoring='accuracy', cv=10, n_jobs=-1, error_score='raise')
sensitivity_score_cv = cross_val_score(knn_clf, descriptors, activities, scoring='recall', cv=10, n_jobs=-1, error_score='raise')
specificity_cv = make_scorer(recall_score, pos_label=0)
specificity_score_cv = cross_val_score(knn_clf, descriptors, activities, scoring=specificity_cv, cv=10, n_jobs=-1, error_score='raise')
balanced_accuracy_score_cv = cross_val_score(knn_clf, descriptors, activities, scoring='balanced_accuracy', cv=10, n_jobs=-1, error_score='raise' )

In [ ]:
f1_score_cv = cross_val_score(knn_clf, descriptors, activities, scoring='f1', cv=10, n_jobs=-1, error_score='raise')
auc_cv = cross_val_score(knn_clf, descriptors, activities, scoring='roc_auc', cv=10, n_jobs=-1, error_score='raise')
precision_score_cv = cross_val_score(knn_clf, descriptors, activities, scoring='precision', cv=10, n_jobs=-1, error_score='raise')
mathews_corrcoef_cv = make_scorer(matthews_corrcoef)
matthews_corrcoef_score_cv = cross_val_score(knn_clf, descriptors, activities, scoring=mathews_corrcoef_cv, cv=10, n_jobs=-1, error_score='raise')

In [ ]:
acc_knn1 = 'Accuracy: %.3f (%.3f)' % (accuracy_score_cv.mean(), accuracy_score_cv.std())
sen_knn1 = 'Sensitivity: %.3f (%.3f)' % (sensitivity_score_cv.mean(), sensitivity_score_cv.std())
spec_knn1 = 'Specificity: %.3f (%.3f)' % (specificity_score_cv.mean(), specificity_score_cv.std())
ba_knn1 = 'Balanced_Accuracy: %.3f (%.3f)' % (balanced_accuracy_score_cv.mean(), balanced_accuracy_score_cv.std())
f1_knn1 = 'F1_score %.3f (%.3f)' % (f1_score_cv.mean(), f1_score_cv.std())
auc_knn1 = 'AUC %.3f (%.3f)' % (auc_cv.mean(), auc_cv.std())
prec_knn1 = 'Precision %.3f (%.3f)' % (precision_score_cv.mean(), precision_score_cv.std())
mcc_knn1 = 'Matthews_corrcoef: %.3f (%.3f)' % (matthews_corrcoef_score_cv.mean(), matthews_corrcoef_score_cv.std())

In [ ]:
import ipywidgets as widgets

def dropdown(change):
    print(change.new)    
    
Buttons = widgets.Dropdown(
    options = [['Accuracy',acc_knn1], ['Sensitivity',sen_knn1], ['Specificity',spec_knn1], ['Balanced Accuracy',ba_knn1],['F1_score',f1_knn1], ['AUC',auc_knn1], ['Precision',prec_knn1], ['MCC',mcc_knn1]],
    description= 'Choose one:',
    disabled=False,
    indent=False
)

Buttons.observe(dropdown, names='value')
display(Buttons)

#### Wrong Predicted Compounds

In [ ]:
from IPython.core.display import display, HTML

knn_predictions = knn_clf.predict(descriptors_test)
indices = [i for i in range(len(activities_test)) if activities_test[i] != knn_predictions[i]]
wrong_predictions_knn = df_Test_Data.iloc[indices,:]
wrong_predicted_compounds = wrong_predictions_knn.index.tolist()
print(str(len((wrong_predicted_compounds))) + ' compounds have been predicted wrong.')
compounds_knn_wrong =  pd.DataFrame(wrong_predictions_knn['ROMol'])

for index, row in compounds_knn_wrong.iterrows():
    display(HTML(str(row['ROMol'])))

#### Save Wrong Predicted Compounds

In [ ]:
PandasTools.WriteSDF(wrong_predictions_knn,"results/BSEP_WP_KNN_Compounds.sdf",properties=list(wrong_predictions_knn.columns))

####  KNN -  Evaluation Test Set

In [ ]:
knn_predict = knn_clf.predict(descriptors_test)

In [ ]:
acc_knn2 = 'Accuracy_score: %.3f' % accuracy_score(np.asarray(activities_test), knn_predict)
sen_knn2 = 'Sensitivity: %.3f ' % recall_score(np.asarray(activities_test), knn_predict)
spec_knn2 = 'Specificity: %.3f ' % recall_score(np.asarray(activities_test), knn_predict, pos_label=0)
ba_knn2 = 'Balanced_accuracy_score: %.3f' % balanced_accuracy_score(np.asarray(activities_test), knn_predict)
f1_knn2 = 'F1_score: %.3f' %  f1_score(np.asarray(activities_test), knn_predict)
auc_knn2 = 'AUC %.3f' % (roc_auc_score(np.asarray(activities_test), knn_predict))
prec_knn2 = 'Precision_score: %.3f' % precision_score(np.asarray(activities_test), knn_predict)
mcc_knn2 = 'Matthews_corrcoef: %.3f' % matthews_corrcoef(np.asarray(activities_test), knn_predict)
rec_knn2 = 'Recall_score: %.3f' % recall_score(np.asarray(activities_test), knn_predict)

In [ ]:
import ipywidgets as widgets

def dropdown(change):
    print(change.new)    
    
Buttons = widgets.Dropdown(
    options = [['Accuracy',acc_rf2], ['Sensitivity',sen_rf2], ['Specificity',spec_rf2], ['Balanced Accuracy',ba_rf2],['F1_score',f1_rf2], ['AUC',auc_rf2], ['Precision',prec_rf2], ['MCC',mcc_rf2],['Recall',rec_rf2]],
    description= 'Choose one:',
    disabled=False,
    indent=False
)

Buttons.observe(dropdown, names='value')
display(Buttons)

In [ ]:
df_Test["Prediction"] = knn_predict
df_Test["Prediction Comparison"] = np.where(pd.to_numeric(df_Test["Classification"]) == df_Test["Prediction"], "Right", "Wrong")
df_Test["Domain"] = df_descriptors_test["LOF"]
df_Test["Domain"] = np.where(df_Test.Domain.values == 1, "In", "Out")
df_KNN_Test = df_Test
df_KNN_Test

In [ ]:
PandasTools.WriteSDF(df_KNN_Test,"results/BSEP_RF_Test_Prediction.sdf",properties=list(df_KNN_Test.columns))
df_KNN_Test.to_csv("results/BSEP_KNN_Test_Prediction.csv")